In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns


import time
import math
import random
import re
import os
import sys


from utils.util import find_files_with_extension
from utils.util import  get_dataframe_from_filepath,calculate_statistics, extract_unique_mbps_and_ms
from utils.util import remove_outliers_iqr, comma_format, create_directory_if_not_exists,print_rtt_stats,save_rtt_stats,return_rtt_stats

In [ ]:
# Load data
mainpth="../data/data_2025-03-19-01-12-25"

graph_directory = os.path.join(mainpth, "Graphs")
create_directory_if_not_exists(graph_directory)

stats_directory = os.path.join(mainpth, "Stats")
create_directory_if_not_exists(stats_directory)


folderpaths = [mainpth+'/client1_data',mainpth+'/client2_data']
filenames, filepaths, filedict = find_files_with_extension(folderpaths, '.log')


In [ ]:
filedict

In [ ]:
folderpaths

In [ ]:
# Call the function and print unique combinations
unique_combinations = extract_unique_mbps_and_ms(folderpaths)
print("Unique Mbps and ms Combinations:")
for mbps, ms in sorted(unique_combinations):
    print(f'{mbps} Mbps, {ms} ms')


In [ ]:
mbps , ms = list(unique_combinations)[0]
net_settings = f'{mbps}_Mbps_{ms}_ms'
print(net_settings)

In [ ]:
df=get_dataframe_from_filepath(filepaths[0])
df['ForeignPort'].unique()

In [ ]:
unique_scenarios_dict={}
for index in range(len(filenames)):
    scenario = "_".join(filenames[index].split('_')[:5])
    if "l4s" in scenario:
        scenario = "_".join(filenames[index].split('_')[:4])
    if scenario in unique_scenarios_dict:
        unique_scenarios_dict[scenario].append(filedict[filenames[index]])
    else:
        unique_scenarios_dict[scenario]=[filedict[filenames[index]]]


In [ ]:
unique_scenarios_dict

In [ ]:
from utils.plotter import  plot_siftr_graph

In [ ]:
for scenario in unique_scenarios_dict.keys():

    print()
    print("Start New Scenario")
    print(scenario)
    print(unique_scenarios_dict[scenario])

    tcp_traffic = unique_scenarios_dict[scenario][0]
    udp_traffic = unique_scenarios_dict[scenario][1]

    print("tcp_traffic",tcp_traffic)
    print("udp_traffic",udp_traffic)


    ttf = get_dataframe_from_filepath(tcp_traffic)
    utf = get_dataframe_from_filepath(udp_traffic)

    ttf = ttf[ttf['ForeignPort'] == "5101"]
    utf = utf[utf['ForeignPort'] == "5101"]

    # Define paths
    paths = {
        "NEWRENO": ttf,
        "UDP": utf,
    }

    print_rtt_stats(paths=paths,scenario=scenario)
    save_rtt_stats(paths=paths,scenario=scenario,stats_directory=stats_directory)



    plot_siftr_graph(paths=paths,
                    ycolumn="SmoothedRTT",
                    title=f"{scenario} SmoothedRTT",
                    xlabel="Time (s)",
                    ylabel="SmoothedRTT (ms)",
                    filename=f'{scenario}_SmoothedRTT',
                    graph_directory=graph_directory,
    )




    plot_siftr_graph(paths=paths,
                    ycolumn="CongestionWindow",
                    title=f"{scenario} CongestionWindow",
                    xlabel="Time (s)",
                    ylabel="Congestion Window (Bytes)",
                    filename=f'{scenario}_CWND',
                    graph_directory=graph_directory,
    )
